### Data preparation

#### Import packages

In [3]:
# Install the needed libraries
import pandas as pd
import numpy as np
from pulp import *
import pulp.apis
from datetime import datetime
import time
import warnings
import yaml
from pandas.core.common import SettingWithCopyWarning

In [4]:
solver_list = pulp.listSolvers(onlyAvailable=True)

In [5]:
solver_list

['PULP_CBC_CMD']

In [419]:
# Ignore SettingWithCopyWarning 
warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)

In [420]:
def load_config():
    try:
        with open('config_file.yaml') as f:
            global constants
            constants = yaml.safe_load(f)
    except OSError as e:
        raise e
    except Exception as e:
        raise e

In [421]:
load_config()

#### Import data

In [422]:
# Import the data
data = pd.read_excel(constants['INPUT_FILE_NAME'], sheet_name='data', index_col=0)
real = pd.read_excel(constants['INPUT_FILE_NAME'], sheet_name='реал', index_col=0)
bandwidth = pd.read_excel(constants['INPUT_FILE_NAME'], sheet_name='ПС', index_col=0)
match = pd.read_excel(constants['INPUT_FILE_NAME'], sheet_name='match', index_col=0)

#### Create dict with variable names

In [423]:
# Create a dict of names on a spreadsheets that we are going to use
names = {"Тариф ж/д" : "t1",
        "Тариф хранение" : "t2",
        "Тариф бренд" : "t3",
        "Тариф ВЛ" : "t4",
        "Плечо, км" : "distance",
        "Дата" : "date",
        "НБ" : "origin",
        "ОУ" : "point",
        "НГ" : "brand",
        "НГ_ПС" : "product",
        "Объем" : "volume"} 

---

**Reorganize the data**

In [424]:
# Rename the names in dataframes
def rename_columns(df_list, names):
    for df in df_list:
        df.rename(columns=names, inplace=True)

# Add the column with the tariff
def add_tariff(df):
    df['cost'] = df['t1'] + df['t2'] + df['t3'] + df['t4']
    df.loc[df['distance'] > 50, 'cost'] = df["t1"] + df["t2"] + df["t3"] +  df["t4"] * df["distance"]

# Make subset for the concrete time period
def select_time_period(df):
    df_subset = df[df['date'] == datetime.strptime(constants['MONTH'], '%Y-%m-%d')]
    return df_subset

# Adjust datatypes
def adjust_datatypes(df_list):
    for df in df_list:
        if 'origin' in df.columns:
            df['origin'] = df['origin'].map(lambda x: int(x.strip('Нефтебаза ')))
        if 'point' in df.columns:
            df['point'] = df['point'].map(lambda x: int(x.strip('АЗС ')))

In [425]:
# Apply changes to dataframes
def apply_changes_dfs(data, real, bandwidth, match, names):
    rename_columns([data, real, bandwidth, match], names)
    add_tariff(data)
    
    data_subset = select_time_period(data)
    real_subset = select_time_period(real)
    bandwidth_subset = select_time_period(bandwidth)

    adjust_datatypes([data_subset,real_subset, bandwidth_subset])
    
    return data_subset, real_subset, bandwidth_subset

In [426]:
data_subset, real_subset, bandwidth_subset = apply_changes_dfs(data, real, bandwidth, match, names)

---

**Create dictionaries for brand-products matches**

In [427]:
# Create brand-product dict
def create_brand_product_dict(df):
    match_brand_product = dict(zip(df['brand'], df['product']))
    
    match_product_brand = dict()
    for key, value in match_brand_product.items():
        match_product_brand.setdefault(value, list()).append(key)
    return match_brand_product, match_product_brand

# Create numeration dict and inverted dict
def create_numeration_dict(df, parameter):
    match_dict = dict(enumerate(df[parameter]))
    inv_dict = {v: k for k, v in match_dict.items()}
    return match_dict, inv_dict

In [428]:
# Create 6 dictionaries (brand-product, brand-number, product-number, and inverted)
match_brand_product, match_product_brand = create_brand_product_dict(match)
match_brand_number, match_number_brand = create_numeration_dict(match,'brand')
match_product_number, match_number_product = create_numeration_dict(match,'product')

---

In [429]:
# Create name of variable
def create_var_Name(row):
    var_name = 'P' + str(int(row["point"])) + 'O' + str(int(row["origin"])) +  '_brand_' + str(int(row["brand_number"]))
    return var_name

# Reorganize the data
def reorganize_df(df, columns_to_keep, match_number_brand, match_number_product):
    df_reorganized = df.reset_index().loc[:, columns_to_keep]
    df_reorganized['brand_number'] = df_reorganized['brand'].map(lambda x: match_number_brand.get(x))
    df_reorganized['product_number'] = df_reorganized['product'].map(lambda x: match_number_product.get(x))
    
    return df_reorganized

In [430]:
columns_to_keep = ['origin', 'point', 'brand', 'product', 'cost']
df_reorganized = reorganize_df(data_subset, columns_to_keep, match_number_brand, match_number_product)

# Create a name variable
df_reorganized['var_Name'] = df_reorganized.apply(lambda row: create_var_Name(row), axis=1) 

---

**Create demand and supply dataframes**

In [431]:
# Create demand df
def create_demand_supply_dfs(real, bandwidth):
    demand = real[['point', 'brand', 'product', 'volume']]
    demand.rename(columns={'volume':'demand'}, inplace=True)
    demand['brand_number'] = demand['brand'].map(lambda x: match_number_brand.get(x))
    demand['product_number'] = demand['product'].map(lambda x: match_number_product.get(x))
    
    supply = bandwidth[['origin', 'product', 'volume']]
    supply.rename(columns={'volume':'supply'}, inplace=True)
    supply['product_number'] = supply['product'].map(lambda x: match_number_product.get(x))
    
    return demand, supply

In [432]:
demand, supply = create_demand_supply_dfs(real_subset, bandwidth_subset)

### Usage of puLP

#### Define function

In [433]:
brands = df_reorganized['brand_number'].unique()

In [434]:
import json

df_reorganized

brand_info_map = {}

def make_brand(brand_id, name):
    return {'name': name, 'id': brand_id, 'product_transportations': {}}

def make_transportation(origin, point, cost, var_name):
    return {'origin': origin, 'point': point, 'cost': cost, 'var_name': var_name}

def add_transportation_to_brand(brand, product_id, transportation):
    if brand['product_transportations'].get(product_id) == None:
        brand['product_transportations'][product_id] = [transportation];
        return brand;
    
    brand['product_transportations'].get(product_id).append(transportation);    

    return brand;


def append_row_to_map(brand_map, row):
    brand_id = row['brand_number'];
    
    if brand_map.get(brand_id) == None:
        brand_map[brand_id] = make_brand(brand_id, row['brand']);
        
    brand_item = brand_map.get(brand_id);
    product_id = row['product_number'];
    
    transportation = make_transportation(row['origin'], row['point'], row['cost'], row['var_Name']);
    brand_map[brand_id] = add_transportation_to_brand(brand_item, product_id, transportation)
    
    return brand_map

for i, row in df_reorganized.iterrows():
    brand_info_map = append_row_to_map(brand_info_map, row);

json_formatted_str = json.dumps(brand_info_map, indent=2, ensure_ascii=False)

In [435]:
def define_problem(df):
    x = [pulp.LpVariable(df['var_Name'][i], lowBound = 0, cat='Continuous') for i in np.arange(0, len(df))]
    # Set function's goal
    problem = pulp.LpProblem('0', sense=LpMinimize)
    # Create a function
    problem += pulp.LpAffineExpression([(x[i], df['cost'][i]) for i in np.arange(0, len(df))])
    x_map = {}
    
    for i in x:
        x_map[str(i)] = i
    
    return problem, x, x_map

In [436]:
# Function for dividing supply between different brands
def merge_supply(brand_number, match_brand_product, match_brand_number, match_product_brand, match_number_brand):
    """This function returns a set of brands' numbers that are produced from the same product"""
    brand_name = match_brand_number[brand_number]
    product_name = match_brand_product[brand_name]
    brands_overlap_names = match_product_brand[product_name]
    brands_overlap_numbers = set(match_number_brand.get(item) for item in brands_overlap_names)

    return brands_overlap_numbers

In [437]:
def extract_unique_points_of_brand(brand):
    points = []
    for product_id in brand.get('product_transportations').keys():
        for transportation in brand.get('product_transportations').get(product_id):
            points.append(transportation.get('point'))

    points = list(set(points))
    
    return points

def extract_unique_origins_of_brand(brand):
    origins = []
    for transportations in brand.get('product_transportations').values():
        for transportation in transportations:
            origins.append(transportation.get('origin'))

    origins = list(set(origins))
    
    return origins

def get_brand_transportations(brand):
    brand_transpotrations = []
    for product_id in brand.get('product_transportations').keys():
        for transportation in brand.get('product_transportations').get(product_id):
            brand_transpotrations.append(transportation)
                
    return brand_transpotrations

def extract_unique_origins(brand_map):
    origins = []
    for brand in brand_map.values():
        origins = origins + extract_unique_origins_of_brand(brand)

    return list(set(origins))

def extract_unique_points(brand_map):
    points = []
    for brand in brand_map.values():
        points = points + extract_unique_points_of_brand(brand)
        
    return list(set(points))

def get_transportations(brand_map):
    transportations = []
    
    for brand in brand_map.values():
        transportations = transportations + get_brand_transportations(brand)
        
    return transportations

In [438]:
def make_restriction_demand(record, problem, demand, x, brands):
    n_constraint = 1
    
    for brand_id in brands:
        brand = record.get(brand_id);
        points = extract_unique_points_of_brand(brand)
    
        for p in points:
            transportations = get_brand_transportations(brand)
            brand_point_transpotrations = [transportation for transportation in transportations if transportation.get('point') == p]
            
            const = float(demand[(demand['point'] == p) & (demand['brand_number'] == brand_id)]['demand'])
            # Add a restriction on demand == const
            problem+= pulp.LpAffineExpression([(x.get(transportation.get('var_name')), 1) for transportation in brand_point_transpotrations ]) == const, str(n_constraint)
            n_constraint +=1
        
    return problem, n_constraint


def make_restriction_supply(problem, record, bandwidth, supply, x, match_brand_product, match_brand_number, match_product_brand, match_number_brand, brands, n_constraint):
    # Set for storing already processed brands 
    brands_accounted_for = set()
    
    # Store origin with infinite supply in a separate list
    origins_with_inf_supply = list(bandwidth[bandwidth['volume'] == np.inf]['origin'].unique())
    
    for b in brands:
        if b in brands_accounted_for:
            continue
        
        brands_overlap_numbers = merge_supply(b, match_brand_product, match_brand_number, match_product_brand, match_number_brand)
        brands_accounted_for.update(brands_overlap_numbers)

        brands_list = [brand for brand in record.values() if brand.get('id') in brands_overlap_numbers]

        origins = []
        for brand in brands_list:
            origins += extract_unique_origins_of_brand(brand)
            
        origins = list(set(origins))
        
        transportations_list = []
        for brand in brands_list:
            transportations_list += get_brand_transportations(brand)

        for o in origins:
            if o in origins_with_inf_supply:
                continue

            origin_transportations = [transportation for transportation in transportations_list if transportation.get('origin') == o]

            product_name = match_brand_product[match_brand_number[b]]
            const = float(supply[(supply['origin'] == o) & (supply['product'] == product_name)]['supply'])

            # Add a restriction on supply <= upper_const
            problem+= pulp.LpAffineExpression([(x.get(i.get('var_name')), 1) for i in origin_transportations ]) <= const, str(n_constraint)
            n_constraint +=1

            # Add a restriction on supply >= lower_const
            const *= constants['PERCENT_LOADING']
            problem+= pulp.LpAffineExpression([(x.get(i.get('var_name')), 1) for i in origin_transportations ]) >= const, str(n_constraint)
            n_constraint +=1 
                    
    return problem, n_constraint

In [439]:
# def make_restriction_demand_df(df, problem, demand, x, brands):
    
#     n_constraint = 1
    
#     for b in brands:
#         df1 = df[df['brand_number'] == b]
#         points = df1['point'].unique()
    
#         for p in points:
#             df2 = df1[df1['point'] == p]
#             const = float(demand[(demand['point'] == p) & (demand['brand_number'] == b)]['demand'])
#             # Add a restriction on demand == const
#             problem+= pulp.LpAffineExpression([(x[i], 1) for i in df2.index ]) == const, str(n_constraint)
#             n_constraint +=1
        
#     return problem, n_constraint

# def make_restriction_supply_df(problem, df, bandwidth, supply, x, match_brand_product, match_brand_number, match_product_brand, match_number_brand, brands, n_constraint):
#     # Set for storing already processed brands 
#     brands_accounted_for = set()
    
#     # Store origin with infinite supply in a separate list
#     origins_with_inf_supply = list(bandwidth[bandwidth['volume'] == np.inf]['origin'].unique())
    
#     for b in brands:
        
#         if b in brands_accounted_for:
#             continue
        
#         else:
#             brands_overlap_numbers = merge_supply(b, match_brand_product, match_brand_number, match_product_brand, match_number_brand)
#             brands_accounted_for.update(brands_overlap_numbers)
        
#             df1 = df[df['brand_number'].isin(brands_overlap_numbers)]
#             origins = df1['origin'].unique()
        
#             for o in origins:
            
#                 if o in origins_with_inf_supply:
#                     continue
#                 else:
                
#                     df2 = df1[df1['origin'] == o]
#                     product_name = match_brand_product[match_brand_number[b]]
#                     const = float(supply[(supply['origin'] == o) & (supply['product'] == product_name)]['supply'])
                
#                     # Add a restriction on supply <= upper_const
#                     problem+= pulp.LpAffineExpression([(x[i], 1) for i in df2.index ]) <= const, str(n_constraint)
#                     n_constraint +=1

#                     # Add a restriction on supply >= lower_const
#                     const *= constants['PERCENT_LOADING']
#                     problem+= pulp.LpAffineExpression([(x[i], 1) for i in df2.index ]) >= const, str(n_constraint)
#                     n_constraint +=1 
                    
#     return problem, n_constraint

In [440]:
# Function for creating dummies names
def create_dummy_Name(df):
    dummies = {}
    
    # Create a name variable
    dummies_names = set(df.apply(lambda row: 'y_P' + str(int(row["point"])) + 'O' + str(int(row["origin"])), axis=1))

    # Create dummy variables and store them in a dict
    for name in dummies_names:
        dummies[name] = pulp.LpVariable(name, lowBound=0, upBound = 1, cat = 'Integer')
        
    return dummies

In [441]:
# def make_restriction_dummy_df(df, problem, n_constraint, dummies, x):
#     points = df['point'].unique()
#     origins = df['origin'].unique()

#     for p in points:
#         for o in origins:
#             df1 = df[(df['point'] == p) & (df['origin'] == o)]
        
#             string = "y_P" + str(p) + "O" + str(o)
#             d = dummies.get(string)
        
#             problem+= pulp.LpAffineExpression([(x[i], 1) for i in df1.index ]) <=  constants['M'] * d, str(n_constraint)
#             n_constraint += 1
            
#     for p in points:
#         trans = []

#         for o in origins:
#             string = "y_P" + str(p) + "O" + str(o)
#             trans.append(dummies.get(string))
            
#         print(trans)

#         summa = sum(np.array(trans))

#         # Add a restriction on dummies
#         problem += summa == 1, str(n_constraint)
#         n_constraint += 1
            
#     return problem

def make_restriction_dummy(record, problem, n_constraint, dummies, x):
    points = extract_unique_points(record)
    origins = extract_unique_origins(record)
    transportations = get_transportations(record)

    for p in points:
        for o in origins:
            po_transportations = [transportation for transportation in transportations if transportation.get('origin') == o and transportation.get('point') == p]
                    
            string = "y_P" + str(p) + "O" + str(o)
            d = dummies.get(string)
        
            problem+= pulp.LpAffineExpression([(x.get(i.get('var_name')), 1) for i in po_transportations ]) <=  constants['M'] * d, str(n_constraint)
            n_constraint += 1
            
    for p in points:
        trans = []

        for o in origins:
            string = "y_P" + str(p) + "O" + str(o)
            trans.append(dummies.get(string))


        summa = sum(np.array(trans))

        # Add a restriction on dummies
        problem += summa == 1, str(n_constraint)
        n_constraint += 1
            
    return problem



In [442]:
dummies = create_dummy_Name(df_reorganized)
problem_defined, x, x_map = define_problem(df_reorganized)
problem, n_constraint = make_restriction_demand(brand_info_map, problem_defined, demand, x_map, brands)
problem, n_constraint = make_restriction_supply(problem, brand_info_map, bandwidth_subset, supply, x_map, match_brand_product, match_brand_number, match_product_brand, match_number_brand, brands, n_constraint)
problem = make_restriction_dummy(brand_info_map, problem, n_constraint, dummies, x_map)

In [443]:
def main(problem):
    start = time.time()
    problem.solve()
    
    # Print the results
    print ("Результат:")
    for variable in problem.variables():
        print (variable.name, "=", variable.varValue)
    print ("Стоимость доставки:")
    print (abs(problem.objective.value()))
    stop = time.time()
    print ("Время :")
    print(stop - start)

In [444]:
if __name__ == '__main__':
    main(problem)

Результат:
P0O0_brand_0 = 0.0
P0O0_brand_1 = 0.0
P0O0_brand_2 = 0.0
P0O0_brand_3 = 0.0
P0O0_brand_4 = 0.0
P0O1_brand_0 = 29.693803
P0O1_brand_1 = 304.57098
P0O1_brand_2 = 281.70649
P0O1_brand_3 = 65.738768
P0O1_brand_4 = 71.355788
P10O0_brand_0 = 42.384663
P10O0_brand_1 = 268.45006
P10O0_brand_2 = 276.05923
P10O0_brand_3 = 67.783332
P10O0_brand_4 = 108.14143
P10O1_brand_0 = 0.0
P10O1_brand_1 = 0.0
P10O1_brand_2 = 0.0
P10O1_brand_3 = 0.0
P10O1_brand_4 = 0.0
P11O0_brand_0 = 18.717185
P11O0_brand_1 = 182.12777
P11O0_brand_2 = 175.19265
P11O0_brand_3 = 40.864389
P11O0_brand_4 = 110.71506
P11O1_brand_0 = 0.0
P11O1_brand_1 = 0.0
P11O1_brand_2 = 0.0
P11O1_brand_3 = 0.0
P11O1_brand_4 = 0.0
P12O0_brand_0 = 29.952396
P12O0_brand_1 = 241.90937
P12O0_brand_2 = 246.86835
P12O0_brand_3 = 54.433692
P12O0_brand_4 = 46.746832
P12O1_brand_0 = 0.0
P12O1_brand_1 = 0.0
P12O1_brand_2 = 0.0
P12O1_brand_3 = 0.0
P12O1_brand_4 = 0.0
P13O0_brand_1 = 150.43719
P13O0_brand_2 = 68.076836
P13O0_brand_3 = 38.306174
P